In [3]:
import argparse
import os
from tqdm import tqdm

import sys
sys.path.append("spacenet_lib")

In [8]:
sys.path

['c:\\Users\\toadi\\Documents\\GitHub\\spacenet_building_detection\\src\\features',
 'c:\\Users\\toadi\\anaconda3\\python39.zip',
 'c:\\Users\\toadi\\anaconda3\\DLLs',
 'c:\\Users\\toadi\\anaconda3\\lib',
 'c:\\Users\\toadi\\anaconda3',
 '',
 'c:\\Users\\toadi\\anaconda3\\lib\\site-packages',
 'c:\\Users\\toadi\\anaconda3\\lib\\site-packages\\win32',
 'c:\\Users\\toadi\\anaconda3\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\toadi\\anaconda3\\lib\\site-packages\\Pythonwin',
 'c:\\Users\\toadi\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\toadi\\.ipython',
 'spacenet_lib']

In [24]:
currentpath = os.getcwd()
print(currentpath)

lev2up = os.path.dirname(os.path.dirname(currentpath))
print(lev2up)

datadir = os.path.join(lev2up,"data")
rasterdir = os.path.join(datadir,"3band")
vectordir = os.path.join(datadir,"processedBuildingLabels\\vectordata")
destdir = os.path.join(datadir,"buildingMaskImages")
print(destdir)

c:\Users\toadi\Documents\GitHub\spacenet_building_detection\src\features
c:\Users\toadi\Documents\GitHub\spacenet_building_detection
c:\Users\toadi\Documents\GitHub\spacenet_building_detection\data\buildingMaskImages


In [ ]:
# Add create_poly_mask

# Reference: 
# https://medium.com/the-downlinq/getting-started-with-spacenet-data-827fd2ec9f53
# https://gist.github.com/avanetten/b295e89f6fa9654c9e9e480bdb2e4d60#file-create_building_mask-py

from osgeo import gdal, ogr
from PIL import Image
import numpy as np
import os


def create_poly_mask(rasterSrc, vectorSrc, npDistFileName='', 
							noDataValue=0, burn_values=1):

	'''
	Create polygon mask for rasterSrc,
	Similar to labeltools/createNPPixArray() in spacenet utilities
	'''
	
	## open source vector file that truth data
	source_ds = ogr.Open(vectorSrc)
	source_layer = source_ds.GetLayer()

	## extract data from src Raster File to be emulated
	## open raster file that is to be emulated
	srcRas_ds = gdal.Open(rasterSrc)
	cols = srcRas_ds.RasterXSize
	rows = srcRas_ds.RasterYSize

	if npDistFileName == '':
		dstPath = ".tmp.tiff"
	else:
		dstPath = npDistFileName

	## create First raster memory layer, units are pixels
	# Change output to geotiff instead of memory 
	memdrv = gdal.GetDriverByName('GTiff') 
	dst_ds = memdrv.Create(dstPath, cols, rows, 1, gdal.GDT_Byte, 
						   options=['COMPRESS=LZW'])
	dst_ds.SetGeoTransform(srcRas_ds.GetGeoTransform())
	dst_ds.SetProjection(srcRas_ds.GetProjection())
	band = dst_ds.GetRasterBand(1)
	band.SetNoDataValue(noDataValue)    
	gdal.RasterizeLayer(dst_ds, [1], source_layer, burn_values=[burn_values])
	dst_ds = 0

	mask_image = Image.open(dstPath)
	mask_image = np.array(mask_image)

	if npDistFileName == '':
		os.remove(dstPath)
		
	return mask_image

In [25]:
def build_labels(rasterdir, vectordir, destdir):
	
	os.makedirs(destdir, exist_ok=True)

	file_count = len([f for f in os.walk(vectordir).__next__()[2] if f[-8:] == ".geojson"])

	print("[INFO] Found {} geojson files. Preparing building mask images...".format(file_count))

	for idx in tqdm(range(1, file_count + 1)):

		src_raster_filename = "3band_AOI_1_RIO_img{}.tif".format(idx)
		src_vector_filename = "Geo_AOI_1_RIO_img{}.geojson".format(idx)

		src_raster_path = os.path.join(rasterdir, src_raster_filename)
		src_vector_path = os.path.join(vectordir, src_vector_filename)
		dst_path = os.path.join(destdir, src_raster_filename)

		create_poly_mask(
			src_raster_path, src_vector_path, npDistFileName=dst_path, 
			noDataValue=0, burn_values=255
		)


if __name__ == "__main__":

	parser = argparse.ArgumentParser()

	parser.add_argument('rasterdir', help='Root directory for raster files (.tif)')
	parser.add_argument('vectordir', help='Root directory for vector files (.geojson)')
	parser.add_argument('destdir', help='Output directory')

	args = parser.parse_args()

	build_labels(args.rasterdir, args.vectordir, args.destdir)

usage: ipykernel_launcher.py [-h] rasterdir vectordir destdir
ipykernel_launcher.py: error: the following arguments are required: rasterdir, vectordir, destdir


SystemExit: 2